In [ ]:
import tensorflow as tf
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig

In [ ]:
path_daisy = 'flower_photos/daisy/*.jpg'
path_dandelion = 'flower_photos/dandelion/*.jpg'
path_roses = 'flower_photos/roses/*.jpg'
path_sunflowers = 'flower_photos/sunflowers/*.jpg'
path_tulips = 'flower_photos/tulipaths_list = [path_daisy, path_dandelion, path_roses, path_sunflowers, path_tulips]ps/*.jpg'


In [ ]:
count = 1
c = 0
for path in paths_list:
    filenames = glob.glob(path)
    for filename in filenames:
        image = cv2.imread(filename)
        gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (128, 128))
        gray_img = cv2.resize(gray_img, (128, 128))
        cv2.imwrite("gray_images/gray_" +str(count) +".jpg", gray_img)
        cv2.imwrite("color_images/color_" +str(count) +".jpg", image)
        count += 1
        c+=1


In [ ]:
filenames = glob.glob("flower_images\*.png")
count = 1
for file in filenames:
    img = cv2.imread(file, 0)
    img = cv2.resize(img, (128, 128))
    cv2.imwrite('actual_gray_test/gray_' +str(count) +'.jpg', img)
    count += 1


In [ ]:
dataset = []

#Read all color images and append into numpy list
for i in range(1, c+1):
    img = cv2.imread("color_images/color_" +str(i) +".jpg" )
    dataset.append(np.array(img))

dataset_source = np.asarray(dataset)
print(dataset_source.shape)

dataset_tar = []

#Read all grayscale images and append into numpy list
for i in range(1, c+1):
    img = cv2.imread("gray_images/gray_" +str(i) +".jpg", 0)
    dataset_tar.append(np.array(img))

dataset_target = np.asarray(dataset_tar)
print(dataset_target.shape)


(0,)
(0,)


In [ ]:
def autoencoder(inputs):

    # Encoder
    net = tf.layers.conv2d(inputs, 128, 2, activation = tf.nn.relu)
    net = tf.layers.max_pooling2d(net, 2, 2, padding = 'same')

    # Decoder
    net = tf.image.resize_nearest_neighbor(net, tf.constant([129, 129]))
    net = tf.layers.conv2d(net, 1, 2, activation = None, name = 'outputOfAuto')

    return net


In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow_addons.optimizers import AdamW

# Define your autoencoder function using Keras layers
def autoencoder(inputs):
    # Encoder
    net = Conv2D(128, (2, 2), activation=tf.nn.relu)(inputs)
    net = MaxPooling2D((2, 2), padding='same')(net)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(net)

    return decoded

ae_inputs = tf.keras.Input(shape=(256, 256, 3), name='inputToAuto', dtype=tf.float32)
ae_target_resized = tf.image.resize(ae_target, (256, 256))  # Resize target to match output size
ae_outputs = autoencoder(ae_inputs)
lr = 0.001 # learning rate

loss = tf.reduce_mean(tf.square(ae_outputs - ae_target))
train_op = AdamW(learning_rate=lr, weight_decay=0.001).minimize(loss, var_list=[ae_inputs, ae_outputs])
init = tf.global_variables_initializer()

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow_addons.optimizers import AdamW

# Define your autoencoder function using Keras layers
def autoencoder(inputs):
    # Encoder
    net = Conv2D(128, (2, 2), activation=tf.nn.relu)(inputs)
    net = MaxPooling2D((2, 2), padding='same')(net)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(net)

    return decoded

ae_inputs = tf.keras.Input(shape=(256, 256, 3), name='inputToAuto', dtype=tf.float32)
ae_target_resized = tf.image.resize(ae_target, (256, 256))  # Resize target to match output size
ae_outputs = autoencoder(ae_inputs)
lr = 0.001 # learning rate

loss = tf.convert_to_tensor(tf.reduce_mean(tf.square(ae_outputs - ae_target)))
train_op = AdamW(learning_rate=lr, weight_decay=0.001).minimize(tf.cast(loss, tf.float32), var_list=[ae_inputs, ae_outputs])
init = tf.global_variables_initializer()

In [ ]:
# define the constant variables
batch_size = 32
epoch_num = 50
saving_path = 'K:/autoencoder_color_to_gray/SavedModel/AutoencoderColorToGray.ckpt'
saver_ = tf.train.Saver(max_to_keep = 3)

# input data to the network is stored in variables
batch_img = dataset_source[0:batch_size]
batch_out = dataset_target[0:batch_size]
num_batches = num_images//batch_size

# Create a session object and run the global variable which was defined earlier
sess = tf.Session()
sess.run(init)

# new images are sent into the network in batches of 32
for ep in range(epoch_num):
    batch_size = 0
    for batch_n in range(num_batches): # batches loop

        # runs the computational graph in the autoencoder with the given input data and target data
        _, c = sess.run([train_op, loss], feed_dict = {ae_inputs: batch_img, ae_target: batch_out})
        print("Epoch: {} - cost = {:.5f}" .format((ep+1), c))
        batch_img = dataset_source[batch_size: batch_size+32]
        batch_out = dataset_target[batch_size: batch_size+32]
        batch_size += 32

    saver_.save(sess, saving_path, global_step = ep)

sess.close()


In [ ]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
saver.restore(sess, 'K:/autoencoder_color_to_gray/SavedModel/AutoencoderColorToGray.ckpt-49')

test_data = []
for file in filenames[0:100]:
    test_data.append(np.array(cv2.imread(file)))

test_dataset = np.asarray(test_data)
batch_imgs = test_dataset
gray_imgs = sess.run(ae_outputs, feed_dict = {ae_inputs: batch_imgs})

for i in range(gray_imgs.shape[0]):
    cv2.imwrite('gen_gray_images/gen_gray_' +str(i) +'.jpeg', gray_imgs[i])
